
<font color="steelblue" size="3" face="微软雅黑">
<h1>Wave Surfing</h1>
<p>根据当日交易量与过去N条交易量的比值，筛选出符合该特征胜率比较高的证券标的，并对这个范围内的标的进行买入和卖出</p>
</font>

In [2]:
# xtquant
from xtquant import xtdata
from xtquant.xttrader import XtQuantTrader
from xtquant import xtconstant
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount

# self defined
import backtest as bt
from backtest import Context 

# data science
import pandas as pd
import numpy as np

# system
import datetime
import time
import logging
import random


# 函数定义

In [4]:
# 获取股票历史交易数据
def get_hist_data(start_date_str, end_date_str, code_list, count, dividend_type='none', period='1d', fields=['open', 'close', 'high', 'low', 'volume', 'amount', 'preClose']):
    xtdata.download_history_data2(code_list, period, start_date_str, end_date_str)
    
    ret = xtdata.get_market_data_ex(field_list=fields,
                                    stock_list=code_list,
                                    period=period,
                                    start_time='',
                                    end_time=end_date_str,
                                    count=count,
                                    dividend_type=dividend_type
            )
    return ret

# 取消订单
def cancel_untraded_order(acc):
    cancelable_order_list = xt_trader.query_stock_orders(acc, cancelable_only = True)
    for cancelable_order in cancelable_order_list:
        cur_time = datetime.datetime.now().timestamp()
        print(cur_time)
        order_time = cancelable_order.order_time
        print(order_time)
        if cur_time - order_time >= 300:
            cancel_result = xt_trader.cancel_order_stock_async(acc, cancelable_order.order_id)

# 获取持仓信息
def get_hold_position():
    # get hold position
    positions = xt_trader.query_stock_positions(account)
    #print("positions:", len(positions))
    #if len(positions) != 0:
    #    print("last position:", positions[-1])
        
    stock_list = []
    position_dict = {}
    for i in range(len(positions)):
        #print("{0} {1} {2} {3} {4} {5}".format(positions[i].account_id, 
        #                           positions[i].stock_code, 
        #                           positions[i].volume, 
        #                           positions[i].open_price, 
        #                           positions[i].market_value, 
        #                           positions[i].avg_price))
        position = {
            "stock_code": positions[i].stock_code, 
            "volume": positions[i].volume, 
            "can_use_volume": positions[i].can_use_volume, 
            "open_price": positions[i].open_price, 
            "market_value": positions[i].market_value, 
            "avg_price": positions[i].avg_price
        }
        stock_list.append(positions[i].stock_code)
        position_dict[positions[i].stock_code] = position
    return stock_list, position_dict

***<font color=steel
    size=5
       face=雅黑>
<mark>xquant trader connect</mark>
</font>***

In [6]:
current_date = datetime.datetime.now()
c_date_str = current_date.strftime("%Y%m%d")
logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levelname)s - %(message)s', filename = c_date_str + '.txt')

In [7]:
class MyXtQuantTraderCallback(XtQuantTraderCallback):
    import logging
    current_date = datetime.datetime.now()
    c_date_str = current_date.strftime("%Y%m%d")
    logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levelname)s - %(message)s', filename = c_date_str + '.txt')
    def on_disconnected(self):
        """
        连接断开
        :return:
        """
        print(datetime.datetime.now(),'连接断开回调')
        connect_result, account = trader_connect()
        while connect_result != 0:
            connect_result = trader_connect()

    def on_stock_order(self, order):
        """
        委托回报推送
        :param order: XtOrder对象
        :return:
        """
        print(datetime.datetime.now(), '委托回调', order.order_remark)
        logging.info('on_stock_order')


    def on_stock_trade(self, trade):
        """
        成交变动推送
        :param trade: XtTrade对象
        :return:
        """
        print(datetime.datetime.now(), '成交回调', trade.order_remark)
        logging.info('on_stock_trade')


    def on_order_error(self, order_error):
        """
        委托失败推送
        :param order_error:XtOrderError 对象
        :return:
        """
        # print("on order_error callback")
        # print(order_error.order_id, order_error.error_id, order_error.error_msg)
        print(f"委托报错回调 {order_error.order_remark} {order_error.error_msg}")
        logging.info('on_order_error')

    def on_cancel_error(self, cancel_error):
        """
        撤单失败推送
        :param cancel_error: XtCancelError 对象
        :return:
        """
        print(datetime.datetime.now(), sys._getframe().f_code.co_name)

    def on_order_stock_async_response(self, response):
        """
        异步下单回报推送
        :param response: XtOrderResponse 对象
        :return:
        """
        print(f"异步委托回调 {response.order_remark}")
        logging.info('on_order_stock_async_response')

    def on_cancel_order_stock_async_response(self, response):
        """
        :param response: XtCancelOrderResponse 对象
        :return:
        """
        print(datetime.datetime.now(), sys._getframe().f_code.co_name)

    def on_account_status(self, status):
        """
        :param response: XtAccountStatus 对象
        :return:
        """
        print(datetime.datetime.now(), sys._getframe().f_code.co_name)

In [8]:
if __name__ == '__main__':
    print("start")
    #指定客户端所在路径
    path = r'D:\国金QMT交易端模拟\bin.x64/../userdata_mini'
    # 生成session id 整数类型 同时运行的策略不能重复
    session_id = int(time.time())
    xt_trader = XtQuantTrader(path, session_id)
    # 开启主动请求接口的专用线程 开启后在on_stock_xxx回调函数里调用XtQuantTrader.query_xxx函数不会卡住回调线程，但是查询和推送的数据在时序上会变得不确定
    # 详见: http://docs.thinktrader.net/vip/pages/ee0e9b/#开启主动请求接口的专用线程
    # xt_trader.set_relaxed_response_order_enabled(True)

    # 创建资金账号为 800068 的证券账号对象
    account = StockAccount('55004948', 'STOCK')
    # 创建交易回调类对象，并声明接收回调
    callback = MyXtQuantTraderCallback()
    xt_trader.register_callback(callback)
    # 启动交易线程
    xt_trader.start()
    # 建立交易连接，返回0表示连接成功
    connect_result = xt_trader.connect()
    print('建立交易连接，返回0表示连接成功', connect_result)
    # 对交易回调进行订阅，订阅后可以收到交易主推，返回0表示订阅成功
    subscribe_result = xt_trader.subscribe(account)
    print('对交易回调进行订阅，订阅后可以收到交易主推，返回0表示订阅成功', subscribe_result)

    #这一行是注册全推回调函数 包括下单判断 安全起见处于注释状态 确认理解效果后再放开
    # xtdata.subscribe_whole_quote(["SH", "SZ"], callback=f)
    # 阻塞主线程退出
    #xt_trader.run_forever()
    # 如果使用vscode pycharm等本地编辑器 可以进入交互模式 方便调试 （把上一行的run_forever注释掉 否则不会执行到这里）
    #interact()

start
建立交易连接，返回0表示连接成功 0
对交易回调进行订阅，订阅后可以收到交易主推，返回0表示订阅成功 0


In [9]:
#connect_result, account, xt_trader = trader_connect()

***<font color=steel
    size=5
       face=雅黑>
<mark>获取目标股票池</mark>
</font>***

In [50]:
#xtdata.download_sector_data()
#sectors = xtdata.get_sector_list()
sector_list = ['创业板', '沪深A股', '沪深ETF', '科创板',
               'ETF主题指数', 'ETF债券型', 'ETF商品型', 
               'ETF股票型', 'ETF行业指数', 'ETF货币型', 'ETF跨境型']
sector_list = ['沪深300']
res = {}
for sector in sector_list:
    stock_list = xtdata.get_stock_list_in_sector(sector)
    for stock in stock_list:
        res[stock] = 1

***<font color=steel
    size=5
       face=雅黑>
<mark>连续多日交易量特征</mark>
</font>***

In [53]:
# datetime formatting
end_date = datetime.datetime.now() + datetime.timedelta(days=1)
end_date_str = end_date.strftime("%Y%m%d")

start_date = end_date - datetime.timedelta(days=60)
start_date_str = start_date.strftime("%Y%m%d")

current_date = datetime.datetime.now()
current_date_str = current_date.strftime("%Y%m%d")

ret = pd.DataFrame()

#获取目标股票池的交易数据
code_list = list(res.keys())
hist_data_total = get_hist_data(start_date_str, end_date_str, code_list, 31)


# 遍历交易数据并进行特征加工

In [55]:
count = 0
for key in hist_data_total.keys():
    count += 1
    if count % 100 == 0:
        print(datetime.datetime.now(), ': ', count)
    code = key.split('.')[0]

    market = key.split('.')[1]

    hist_data = hist_data_total[key]
   
    if hist_data.shape[0] < 16:
        continue
        
    stat = hist_data.iloc[-1].to_dict()

    # 针对volume进行特征加工
    va1 = hist_data['volume'].iloc[-2]
    stat['va1'] = va1
    
    va2 = hist_data['volume'].iloc[-3]
    stat['va2'] = va2

    va2_6 = hist_data['volume'][-7:-2].mean()
    stat['va2_6'] = va2_6

    va3_7 = hist_data['volume'][-8:-3].mean()
    stat['va3_7'] = va3_7
    
    va1_5 = hist_data['volume'][-6:-1].mean()
    stat['va1_5'] = va1_5

    va5_10 = hist_data['volume'][-11:-6].mean()
    stat['va5_10'] = va5_10
    
    va5_15 = hist_data['volume'][-16:-6].mean()
    stat['va5_15'] = va5_15

    # 针对amount进行特征加工
    amo1 = hist_data['amount'].iloc[-2]
    stat['amo1'] = amo1

    amo2 = hist_data['amount'].iloc[-3]
    stat['amo2'] = amo2

    amo2_6 = hist_data['amount'][-7:-2].mean()
    stat['amo2_6'] = amo2_6

    amo3_7 = hist_data['amount'][-8:-3].mean()
    stat['amo3_7'] = amo3_7

    amo1_5 = hist_data['amount'][-6:-1].mean()
    stat['amo1_5'] = amo1_5

    amo5_10 = hist_data['amount'][-11:-6].mean()
    stat['amo5_10'] = amo5_10

    amo5_15 = hist_data['amount'][-16:-6].mean()
    stat['amo5_15'] = amo5_15
    
    df = pd.DataFrame(data=stat, index=[key])
    
    ret = pd.concat([df, ret])

2025-05-23 21:25:57.524755 :  100
2025-05-23 21:25:57.920788 :  200
2025-05-23 21:25:59.885022 :  300


# Calculate statistic features

In [58]:
ret['va1_5_va5_10'] = ret['va1_5'] / ret['va5_10']
ret['va1_5_va5_15'] = ret['va1_5'] / ret['va5_15']
ret['va0_va1_5'] = ret['volume'] / ret['va1_5']
ret['va1_va2_6'] = ret['va1'] / ret['va2_6']
ret['va2_va3_7'] = ret['va2'] / ret['va3_7']
ret['amo1_5_amo5_10'] = ret['amo1_5'] / ret['amo5_10']
ret['amo1_5_amo5_15'] = ret['amo1_5'] / ret['amo5_15']
ret['amo0_amo1_5'] = ret['amount'] / ret['amo1_5']
ret['amo1_amo2_6'] = ret['amo1'] / ret['amo2_6']
ret['amo2_amo3_7'] = ret['amo2'] / ret['amo3_7']


# 基本面特征

In [60]:
def get_inst_detail(code):
    return xtdata.get_instrument_detail(code)

ret['inst_detail'] = ret.index.map(get_inst_detail)
ret.to_csv('inst_detail.csv', index=False)
ret = ret.dropna()
ret['inst_name'] = ret['inst_detail'].apply(lambda x: x['InstrumentName'])
# remove ST instrument
ret = ret[ret['inst_detail'].apply(lambda x: 'ST' not in x['InstrumentName'])]
# remove non-ETF instrument which is less than 1 dollar
ret = ret[(
    (ret['inst_detail'].apply(lambda x: 'ETF' not in x['InstrumentName'])) & (ret['close'] > 1)
) | (ret['inst_detail'].apply(lambda x: 'ETF' in x['InstrumentName']))]


In [63]:
#get_inst_detail('301269.SZ')
#ret['inst_name']
ret.head(10)
#ret['inst_name'] = ret['inst_detail'].apply(lambda x: x['InstrumentName'])
ret['inst_detail'].apply(lambda x: x['InstrumentName'])

301269.SZ    华大九天
300999.SZ     金龙鱼
300979.SZ    华利集团
300896.SZ     爱美客
300832.SZ     新产业
             ... 
600015.SH    华夏银行
600011.SH    华能国际
600010.SH    包钢股份
600009.SH    上海机场
600000.SH    浦发银行
Name: inst_detail, Length: 300, dtype: object

***<font color=steel
    size=5
       face=雅黑>
<mark>买入</mark>
</font>***

In [28]:
# 读取成交记录
current_date = datetime.datetime.now() - datetime.timedelta(days=1)
c_date_str = current_date.strftime("%Y%m%d")
try:
    trade_df = pd.read_csv('trade_record_' + c_date_str + '.csv')
    trade_list = trade_df['stock_code'].to_list()
except:
    trade_list = []
trade_list

[]

In [29]:
#while True:
#    hour = datetime.datetime.now().hour
#    minute = datetime.datetime.now().minute
#    cur_time = hour * 100 + minute
#    logging.info("check time: " + str(cur_time))
#    if cur_time < 930:
#        logging.info("NOT TRADING TIME")
#        time.sleep(60)
#        continue
#    else:
#        logging.info("START BUYING")
#        break


In [30]:
#TODO: 当日已卖出不再买入 √
#TODO: 已持有股票不再买入 √
#TODO: order_target_value √
#TODO: 下跌过程中放量
#TODO：保存成交历史 √
#TODO: 剔除停牌证券 √
#TODO: 降低行业集中度
current_date = datetime.datetime.now() #+ datetime.timedelta(days=1)
c_date_str = current_date.strftime("%Y%m%d")
print(c_date_str)
#ret.to_csv('test_data_'+c_date_str+'.csv')
#ret = pd.read_csv('test_data_'+c_date_str+'.csv',index_col=0)
######################STRATEGY########################################
# ETF close price may less than 1, otherwise must bigger than 1
stock_df = ret[(ret['va0_va1_5'] >= 3) & (ret['va1_5'] >= 100000)].sort_values(['va0_va1_5', 'va1_5_va5_10'], ascending=False)
######################################################################
stock_list = stock_df.index.to_list()
print('stock_list: ', stock_list)

bought_stock_list, position_dict = get_hold_position()
print('bought_stock_list: ', bought_stock_list)
print('trade_list: ', trade_list)

current_date = datetime.datetime.now() + datetime.timedelta(days=1)
c_date_str = current_date.strftime("%Y%m%d")
print(c_date_str)
order_list = []
for stock in stock_list:
    print(stock)
    code_list = [stock]
    openInt = xtdata.get_full_tick(code_list)[stock]['openInt']
    print(openInt)
    if openInt == 1:
        continue
    if stock not in bought_stock_list and stock not in trade_list:
        order_list.append(stock)
        
#stock_list = ['588680.SH']
print("order_list: ", order_list)
total_value = 200000
if len(order_list) == 0:
    pass
else:
    order_value = int(total_value/len(order_list))
    print(order_value)
    for stock in order_list:
        #TODO：get_market_data_ex
        code = stock.split('.')[0]
        #print(code)
        market = stock.split('.')[1]
        
        #print(xtdata.get_full_tick(code_list))
        global context
        context = Context(market, 100000000, c_date_str, c_date_str)
        context.bkt_dt = current_date.strftime("%Y%m%d")
        hist_data = bt.get_hist_data_by_count(context, code, 1, 'none', 'tick', ['lastPrice'])
        #logging.info(hist_data)
        #print(hist_data)
        last_price = hist_data.values[0][0]
        print('last_price: ', last_price)
        if last_price <= 0:
            continue
        amount = int(order_value / last_price)
        print('order amount: ', amount)
        if amount % 100 != 0:
            amount = int(amount / 100) * 100
            logging.info('adjusted to buyable amount ' + str(amount))
            
        print("#############actual order: ", stock)
        async_seq = xt_trader.order_stock_async(account, 
                                                    stock, 
                                                    xtconstant.STOCK_BUY, 
                                                    amount, 
                                                    xtconstant.FIX_PRICE, 
                                                    last_price, 
                                                    'strategy_buy', 
                                                    stock)
    

20250523
stock_list:  ['300450.SZ']
bought_stock_list:  ['510310.SH', '510360.SH', '513000.SH', '513910.SH', '515180.SH', '515300.SH', '600015.SH', '600023.SH', '600150.SH', '600219.SH', '600372.SH', '600760.SH', '600887.SH', '600893.SH', '601006.SH', '601111.SH', '601169.SH', '601318.SH', '601319.SH', '601878.SH', '601989.SH', '688009.SH', '000157.SZ', '000768.SZ', '000776.SZ', '001213.SZ', '001979.SZ', '300015.SZ', '300308.SZ']
trade_list:  []
20250524
300450.SZ
12
order_list:  ['300450.SZ']
200000
last_price:  0.0


In [66]:
ret.sort_values(
    ['va0_va1_5', 'va1_5', 'va1_5_va5_10'], ascending=False
)[['va0_va1_5', 'va1_5', 'va1_5_va5_10', 'inst_name', 'inst_detail']].head(10)


,va0_va1_5,va1_5,va1_5_va5_10,inst_name,inst_detail
300418.SZ,9.747780,155602.4,0.653262,昆仑万维,"{'ExchangeID': 'SZ', 'InstrumentID': '300418',..."
600460.SH,3.686632,106891.6,0.696675,士兰微,"{'ExchangeID': 'SH', 'InstrumentID': '600460',..."
002252.SZ,2.799802,202545.4,0.818354,上海莱士,"{'ExchangeID': 'SZ', 'InstrumentID': '002252',..."
002074.SZ,2.489453,754465.6,1.196560,国轩高科,"{'ExchangeID': 'SZ', 'InstrumentID': '002074',..."
600745.SH,2.477753,124355.4,0.961973,闻泰科技,"{'ExchangeID': 'SH', 'InstrumentID': '600745',..."
000625.SZ,2.322502,556747.0,1.369691,长安汽车,"{'ExchangeID': 'SZ', 'InstrumentID': '000625',..."
601998.SH,2.136395,444233.8,0.742036,中信银行,"{'ExchangeID': 'SH', 'InstrumentID': '601998',..."
601766.SH,2.122384,450398.6,0.646677,中国中车,"{'ExchangeID': 'SH', 'InstrumentID': '601766',..."
002028.SZ,2.050156,27677.4,0.533631,思源电气,"{'ExchangeID': 'SZ', 'InstrumentID': '002028',..."
001965.SZ,2.043460,127643.8,0.823320,招商公路,"{'ExchangeID': 'SZ', 'InstrumentID': '001965',..."


In [68]:
ret.sort_values(
    ['va0_va1_5', 'va1_5','va1_5_va5_10'], ascending=False
)[['va0_va1_5', 'va1_5','va1_5_va5_10', 'inst_name', 'inst_detail']].tail(10)

,va0_va1_5,va1_5,va1_5_va5_10,inst_name,inst_detail
300033.SZ,0.529500,52963.2,0.540075,同花顺,"{'ExchangeID': 'SZ', 'InstrumentID': '300033',..."
000596.SZ,0.518688,24673.8,1.147309,古井贡酒,"{'ExchangeID': 'SZ', 'InstrumentID': '000596',..."
600183.SH,0.507878,233822.0,0.667149,XD生益科,"{'ExchangeID': 'SH', 'InstrumentID': '600183',..."
300832.SZ,0.499274,51234.4,1.284173,新产业,"{'ExchangeID': 'SZ', 'InstrumentID': '300832',..."
000776.SZ,0.499094,556465.8,0.824465,广发证券,"{'ExchangeID': 'SZ', 'InstrumentID': '000776',..."
688303.SH,0.497656,86017.2,0.538832,大全能源,"{'ExchangeID': 'SH', 'InstrumentID': '688303',..."
601799.SH,0.471207,11763.4,0.979500,星宇股份,"{'ExchangeID': 'SH', 'InstrumentID': '601799',..."
601100.SH,0.459542,108366.6,1.225215,恒立液压,"{'ExchangeID': 'SH', 'InstrumentID': '601100',..."
000425.SZ,0.444547,737555.4,0.894218,徐工机械,"{'ExchangeID': 'SZ', 'InstrumentID': '000425',..."
601728.SH,0.434377,1105078.2,1.208230,中国电信,"{'ExchangeID': 'SH', 'InstrumentID': '601728',..."


In [70]:
hist_data_total['600760.SH']

,open,high,low,close,volume,amount,preClose
20250407,41.68,42.63,39.06,41.24,346522,1.428184e+09,42.94
20250408,41.22,42.00,41.13,42.00,220872,9.174035e+08,41.24
20250409,41.91,44.81,41.66,44.27,457739,2.008231e+09,42.00
20250410,44.26,45.10,43.57,44.60,345152,1.534035e+09,44.27
20250411,44.60,44.83,44.03,44.83,211089,9.383244e+08,44.60
20250414,44.83,44.95,44.16,44.73,161586,7.214135e+08,44.83
20250415,44.50,44.51,43.80,44.00,139512,6.139293e+08,44.73
20250416,44.70,44.75,43.58,44.48,143286,6.309853e+08,44.00
20250417,44.10,44.60,43.96,44.20,130769,5.785480e+08,44.48
20250418,43.93,44.18,43.31,44.04,137804,6.033030e+08,44.20


***<font color=steel
    size=5
       face=雅黑>
<mark>卖出</mark>
</font>***

In [35]:
#TODO：上涨过程中突然跌量
#TODO: 连续两天成交量是前5天的3倍，提高卖出上线至15%，20%，25%不等
#TODO: 持有5天以上，收益率为正
def check_for_selling(stock_list, position_dict):
    import backtest as bt
    from backtest import Context 
    import pandas as pd
    count = 0
    import datetime
    current_date = datetime.datetime.now() + datetime.timedelta(days=1)
    #current_date = datetime.datetime.strptime('20240621', "%Y%m%d")
    c_date_str = current_date.strftime("%Y%m%d")
    #TODO: get accurate start date
    start_date = current_date
    s_date_str = start_date.strftime("%Y%m%d")
    ret = pd.DataFrame()
    #stock_list = ['517520.SH']
    end_date = datetime.datetime.now() + datetime.timedelta(days=1)
    #current_date = datetime.datetime.strptime('20240621', "%Y%m%d")
    end_date_str = end_date.strftime("%Y%m%d")
    #TODO: get accurate start date
    start_date = end_date - datetime.timedelta(days=1)
    start_date_str = start_date.strftime("%Y%m%d")
    for key in stock_list:
        if position_dict[key]['can_use_volume'] == 0:
            continue
        logging.info("stock code: " + key)
        code = key.split('.')[0]
        print(code)
        market = key.split('.')[1]
        print(market)
        global context
        context = Context(market, 100000000, c_date_str, c_date_str)
        context.bkt_dt = current_date.strftime("%Y%m%d")
        #TODO：get_market_data_ex
        #hist_data = bt.get_hist_data_by_count(context, code, 1, 'none', 'tick', ['lastPrice'])
        code_list = [key]
        hist_data = get_hist_data(start_date_str, end_date_str, code_list, 1, 'none','tick', ['lastPrice'])
        #hist_data = hist_data[key]
        logging.info(hist_data)
        if hist_data[key].shape[0] == 0:
            continue
        last_price = hist_data[key].values[0][0]
        logging.info("last_price: " + str(last_price))
        avg_price = position_dict[key]['avg_price']
        logging.info("avg_price: " + str(avg_price))
        
        roe = (last_price / avg_price) - 1
        logging.info("roe: " + str(roe))
        #TODO: 创业板涨跌幅为20%~-10%，其他为10%~-5%
        #if roe >= 0.1  or roe <= -0.05:
        if roe <= -0.05:
            logging.warning("sell " + str(position_dict[key]['can_use_volume']) + " shares of " + key)
            async_seq = xt_trader.order_stock_async(account, 
                                                    key, 
                                                    xtconstant.STOCK_SELL, 
                                                    position_dict[key]['can_use_volume'], 
                                                    xtconstant.FIX_PRICE,
                                                    last_price,
                                                    'strategy_sell'
                                                    )

# main selling while loop

In [37]:
while True:
    hour = datetime.datetime.now().hour
    minute = datetime.datetime.now().minute
    cur_time = hour * 100 + minute
    logging.info("check time: " + str(cur_time))
    if cur_time < 930 or (cur_time > 1130 and cur_time < 1300):
        logging.info("NOT TRADING TIME")
        time.sleep(60)
        continue
    if  cur_time > 1530:
        break
    cancel_untraded_order(account)
    stock_list, position_dict = get_hold_position()
    check_for_selling(stock_list, position_dict)
    time.sleep(60)

510310
SH
510360
SH
513000
SH
513910
SH
515180
SH
515300
SH
600015
SH
600023
SH
600150
SH
600219
SH
600372
SH
600760
SH
600887
SH
600893
SH
601006
SH
601111
SH
601169
SH
601318
SH
601319
SH
601878
SH
601989
SH
688009
SH
000157
SZ
000768
SZ
000776
SZ
001213
SZ
001979
SZ
300015
SZ
300308
SZ
510310
SH
510360
SH
513000
SH
513910
SH
515180
SH
515300
SH
600015
SH
600023
SH
600150
SH
600219
SH
600372
SH
600760
SH
600887
SH
600893
SH
601006
SH
601111
SH
601169
SH
601318
SH
601319
SH
601878
SH
601989
SH
688009
SH
000157
SZ
000768
SZ
000776
SZ
001213
SZ
001979
SZ
300015
SZ
300308
SZ
510310
SH
510360
SH
513000
SH
513910
SH
515180
SH
515300
SH
600015
SH
600023
SH
600150
SH
600219
SH
600372
SH
600760
SH
600887
SH
600893
SH
601006
SH
601111
SH
601169
SH
601318
SH
601319
SH
601878
SH
601989
SH
688009
SH
000157
SZ
000768
SZ
000776
SZ
001213
SZ
001979
SZ
300015
SZ
300308
SZ
510310
SH
510360
SH
513000
SH
513910
SH
515180
SH
515300
SH
600015
SH
600023
SH
600150
SH
600219
SH
600372
SH
600760
SH
600887
SH


***<font color=steel
    size=5
       face=雅黑>
<mark>当日成交记录存储</mark>
</font>***

In [39]:
def record_today_trades():
    trades = xt_trader.query_stock_trades(account)
    col_list = ['account_type',
               'account_id',
               'stock_code',
               'order_type',
               'traded_id',
               'traded_time ',
               'traded_price',
               'traded_volume',
               'traded_amount',
               'order_id',
               'order_sysid ',
               'strategy_name',
               'order_remark',
               'direction',
               'offset_flag']
    
    data = []
    
    for trade in trades:
        col_data = [
                trade.account_type,
                trade.account_id,
                trade.stock_code,
                trade.order_type,
                trade.traded_id,
                trade.traded_time,
                trade.traded_price,
                trade.traded_volume ,
                trade.traded_amount ,
                trade.order_id,
                trade.order_sysid,
                trade.strategy_name ,
                trade.order_remark,
                trade.direction,
                trade.offset_flag
                ]
        data.append(col_data)
    current_date = datetime.datetime.now()
    c_date_str = current_date.strftime("%Y%m%d")
    #c_date_str = '20250515'
    trades_df = pd.DataFrame(data, columns=col_list)
    trades_df.to_csv('trade_record_' + c_date_str + '.csv', index=False)
record_today_trades()